## Data Preparation stage

##### Join raw accident data with fault DB, update parameters

In [1]:
!export PYTHONPATH=$PYTHONPATH:~/src

'export' is not recognized as an internal or external command,
operable program or batch file.


In [1]:
import pandas as pd
import numpy as np
from tqdm import tqdm
# from src.data.data_prep import convert_cols_type

In [2]:
def convert_cols_type(df: pd.DataFrame, features_l: list, type_: str):
    for feature in features_l:
        df[feature] = df[feature].astype(type_)
    return df

In [3]:
cat_features = [
    'circumstances',
    'road_signs',
    'road_surface',
    'visibility',
    'road_conditions'
]

In [4]:
df_fil = pd.read_csv('../data/interim/data_bivac_filtered.csv')
print(f"Data shape: {df_fil.shape}")
print(f"N accidents after filtering: {df_fil.accident_id.nunique()}")

Data shape: (18444, 11)
N accidents after filtering: 4217


In [5]:
df_fau = pd.read_excel('../data/raw/fault with param.xlsx', engine='openpyxl')
print(f"Data shape: {df_fau.shape}")
print(f"N accidents after in Fault DB: {df_fau.accident_id.nunique()}")

Data shape: (6252, 9)
N accidents after in Fault DB: 1173


In [6]:
cat_features_fil = [
    'accident_id', 
    'vehicle_id', 
    'circumstances', 
    'road_signs',
    'road_surface', 
    'visibility', 
    'road_conditions',
    'address_id'
    ]

df_fil.accident_date = pd.to_datetime(df_fil.accident_date)

df_fil = convert_cols_type(
    df=df_fil,
    features_l=cat_features_fil,
    type_='str'
)

In [7]:
cat_features_fau = [
    'accident_id', 
    'accident_vehicle_id', 
    'circumstances', 
    'road_signs',
    'road_surface', 
    'visibility', 
    'road_conditions'
    ]

df_fau = convert_cols_type(
    df=df_fau,
    features_l=cat_features_fau,
    type_='str'
)

In [8]:
print(f"{df_fau.accident_id.nunique()} accidents in Fault DB")
print(f"All accidents from Fault DB are in our filtered data: {df_fil[df_fil.accident_id.isin(df_fau.accident_id.unique())].accident_id.unique() in df_fau.accident_id.unique()}")

1173 accidents in Fault DB
All accidents from Fault DB are in our filtered data: True


In [9]:
df_fau.type.value_counts()

FP    5246
FS     868
FA     138
Name: type, dtype: int64

In [10]:
df_fau.columns

Index(['accident_vehicle_id', 'accident_id', 'type', 'case', 'circumstances',
       'road_signs', 'road_surface', 'visibility', 'road_conditions'],
      dtype='object')

In [11]:
df_fau[df_fau.accident_id=="16141"]

,accident_vehicle_id,accident_id,type,case,circumstances,road_signs,road_surface,visibility,road_conditions
0,34467,16141,FP,FG,2.0,nan,nan,nan,nan
1,34467,16141,FP,FG,12.0,nan,nan,nan,nan
2,34468,16141,FP,FG,2.0,nan,nan,nan,nan
3,34468,16141,FP,FG,12.0,nan,nan,nan,nan


In [12]:
df_fil[df_fil.accident_id=="16141"]

,compensation_status,accident_id,vehicle_id,fd_decision,circumstances,road_signs,road_surface,visibility,road_conditions,accident_date,address_id
0,VALIDATED_EIF,16141,26245.0,NG,nan,nan,1.0,2.0,1.0,2020-02-01 01:30:00+04:00,141147
1,VALIDATED_EIF,16141,26244.0,FG,2.0,nan,1.0,2.0,1.0,2020-02-01 01:30:00+04:00,141147
2,VALIDATED_EIF,16141,26244.0,FG,12.0,nan,1.0,2.0,1.0,2020-02-01 01:30:00+04:00,141147
3,VALIDATED_EIF,16141,26244.0,FG,35.0,nan,1.0,2.0,1.0,2020-02-01 01:30:00+04:00,141147


In [13]:
nan_features_vehicles = []
for vehicle in tqdm(df_fau.accident_vehicle_id.unique()):
    if (df_fau[df_fau.accident_vehicle_id==vehicle][['road_surface', 'visibility', 'road_conditions']]=='nan').all().all():
        nan_features_vehicles.append(vehicle)
        
nan_features_accidents = list(df_fau[df_fau.accident_vehicle_id.isin(nan_features_vehicles)].accident_id.unique())

100%|█████████████████████████████████████████████████████████████████████████████| 2493/2493 [00:02<00:00, 889.87it/s]


In [14]:
print(f'We have {len(nan_features_accidents)} accidents that dont have features info in fault db but have in initial table')

We have 854 accidents that dont have features info in fault db but have in initial table


In [17]:
# I'm replacing NAN features fot these accidents with the features fromm our DB
df_fau_wof = df_fau[df_fau.accident_id.isin(nan_features_accidents)]
df_fau_p_wof = df_fau_wof.pivot_table(['case',
                                ], ['accident_vehicle_id', 'accident_id', 'circumstances',
                                'road_signs',
                                'road_surface',
                                'visibility',
                                'road_conditions'], 'type', aggfunc='first').reset_index()
df_fau_p_wof.columns = df_fau_p_wof.columns.map(''.join)
df_fau_p_wof.caseFA = df_fau_p_wof.caseFA.fillna(df_fau_p_wof.caseFP)
df_fau_p_wof.caseFS = df_fau_p_wof.caseFS.fillna(df_fau_p_wof.caseFA)

df_fau_p_wof['fd_decision'] = df_fau_p_wof.caseFS

print(df_fau_p_wof.accident_id.nunique())
df_fau_p_wof[['accident_vehicle_id', 'accident_id', 'fd_decision']].drop_duplicates()

854


,accident_vehicle_id,accident_id,fd_decision
0,100003,47319,NG
2,100004,47319,FG
4,100025,47330,NG
6,100026,47330,FG
9,100052,47343,NG
...,...,...,...
4111,99429,47041,NG
4113,99850,47248,NG
4115,99851,47248,FG
4117,99871,47258,FG


In [20]:
fil_a = df_fil[df_fil.accident_id.isin(nan_features_accidents)][['vehicle_id', 'accident_id', 'fd_decision']].drop_duplicates()
fil_a.merge(df_fau_p_wof[['accident_vehicle_id', 'accident_id', 'fd_decision']].drop_duplicates(), on='accident_id')

,vehicle_id,accident_id,fd_decision_x,accident_vehicle_id,fd_decision_y
0,26245.0,16141,NG,34467,FG
1,26245.0,16141,NG,34468,FG
2,26244.0,16141,FG,34467,FG
3,26244.0,16141,FG,34468,FG
4,26320.0,16266,FG,34716,FG
...,...,...,...,...,...
3861,95080.0,60538,FG,127816,FG
3862,95368.0,60595,FG,127932,NG
3863,95368.0,60595,FG,127933,FG
3864,95366.0,60595,NG,127932,NG


In [28]:
# I'm filtering out those accidnets

df_fau_wf = df_fau[~df_fau.accident_id.isin(nan_features_accidents)]

print(f"{df_fau_wf.accident_id.nunique()} accidents from Fault DB are being considered")

In [25]:
# getting the last fg-ng status per vehicle from fault db

df_fau_p = df_fau_wf.pivot_table(['case',
                                ], ['accident_vehicle_id', 'accident_id', 'circumstances',
                                'road_signs',
                                'road_surface',
                                'visibility',
                                'road_conditions'], 'type', aggfunc='first').reset_index()
df_fau_p.columns = df_fau_p.columns.map(''.join)
df_fau_p.caseFA = df_fau_p.caseFA.fillna(df_fau_p.caseFP)
df_fau_p.caseFS = df_fau_p.caseFS.fillna(df_fau_p.caseFA)

df_fau_p['fd_decision'] = df_fau_p.caseFS

df_fau_p

,accident_vehicle_id,accident_id,circumstances,road_signs,road_surface,visibility,road_conditions,caseFA,caseFP,caseFS,fd_decision
0,100569,47590,12.0,nan,1.0,2.0,1.0,FG,FG,FG,FG
1,100569,47590,15.0,nan,1.0,2.0,1.0,FG,FG,FG,FG
2,100569,47590,2.0,nan,1.0,2.0,1.0,FG,FG,FG,FG
3,100569,47590,21.0,nan,1.0,2.0,1.0,FG,FG,FG,FG
4,100569,47590,25.0,nan,1.0,2.0,1.0,FG,FG,FG,FG
...,...,...,...,...,...,...,...,...,...,...,...
2154,99933,47288,4.0,163.0,1.0,1.0,1.0,NG,NG,NG,NG
2155,99933,47288,4.0,31.0,1.0,1.0,1.0,NG,NG,NG,NG
2156,99933,47288,35.0,13.0,1.0,1.0,1.0,NG,NG,NG,NG
2157,99933,47288,35.0,163.0,1.0,1.0,1.0,NG,NG,NG,NG


In [28]:
df_fau_p[df_fau_p.accident_id.isin(nan_features_accidents)].accident_id.nunique()

8

In [22]:
print(f"There are {df_fau_p[pd.isna(df_fau_p.caseFS)].accident_id.nunique()} accidents in Fault DB with unknown FG/NG status")

There are 0 accidents in Fault DB with unknown FG/NG status


In [52]:
df_faulted_accidents = df_fil[df_fil.accident_id.isin(df_fau_p.accident_id.unique())]
dict_acc_date = dict(zip(df_faulted_accidents.accident_id, df_faulted_accidents.accident_date))

df_fau_p['accident_date'] = df_fau_p.accident_id.map(dict_acc_date)
df_fau_p = df_fau_p.rename(columns={'accident_vehicle_id': 'vehicle_id'})

df = pd.concat([
    df_fil[~df_fil.accident_id.isin(df_fau_p.accident_id.unique())], 
    df_fau_p[['accident_id', 'vehicle_id', 'fd_decision',
              'circumstances', 'road_signs', 'road_surface', 'visibility',
              'road_conditions', 'accident_date']]])

df = df.sort_values(by=['accident_date', 'accident_id'])
df

,compensation_status,accident_id,vehicle_id,fd_decision,circumstances,road_signs,road_surface,visibility,road_conditions,accident_date,address_id
11123,VALIDATED_EIF,84752,138906.0,FG,2.0,nan,5.0,1.0,1.0,2018-01-11 08:10:00+04:00,754074
11124,VALIDATED_EIF,84752,138906.0,FG,12.0,nan,5.0,1.0,1.0,2018-01-11 08:10:00+04:00,754074
13205,VALIDATED_EIF,101310,168010.0,FG,4.0,nan,1.0,1.0,1.0,2018-06-21 12:30:00+04:00,910393
13206,VALIDATED_EIF,101310,168010.0,FG,13.0,nan,1.0,1.0,1.0,2018-06-21 12:30:00+04:00,910393
13207,VALIDATED_EIF,101310,168010.0,FG,25.0,nan,1.0,1.0,1.0,2018-06-21 12:30:00+04:00,910393
...,...,...,...,...,...,...,...,...,...,...,...
18439,VALIDATED_EIF,188892,566531.0,NG,nan,nan,1.0,1.0,1.0,2021-07-19 01:40:00+04:00,1968163
18440,VALIDATED_EIF,188892,566529.0,FG,9.0,nan,1.0,1.0,1.0,2021-07-19 01:40:00+04:00,1968163
18441,VALIDATED_EIF,188892,566529.0,FG,11.0,nan,1.0,1.0,1.0,2021-07-19 01:40:00+04:00,1968163
18442,VALIDATED_EIF,188892,566529.0,FG,17.0,nan,1.0,1.0,1.0,2021-07-19 01:40:00+04:00,1968163


In [24]:
df.accident_id.value_counts()

16310    28
41262    26
39311    24
59635    24
32270    24
         ..
69516     2
83926     2
75846     2
81030     2
83344     2
Name: accident_id, Length: 4217, dtype: int64

In [33]:
df[df.accident_id=='59635'].accident_date.nunique()

# so there's no accident id for diffent dates

1

In [36]:
df.fd_decision.isna().any()

# there's no unknown fd decision

False

In [37]:
df.shape

(19091, 11)

### Get categories names

In [53]:
l_category_decoding = []
for sheet_name in tqdm(cat_features):
    df_temp = pd.read_excel('../data/raw/data with translations.xlsx', sheet_name=sheet_name)
    dict_temp = dict(zip(df_temp.iloc[:, 0].astype(str) + '.0', df_temp.iloc[:, 1]))
    df[f"{sheet_name}_id"] = df[sheet_name]
    df[sheet_name] = df[f"{sheet_name}_id"].map(dict_temp)

    l_category_decoding.append({sheet_name: dict_temp})
    
print(df.shape)
df.head()

100%|█████████████████████████████████████████████| 5/5 [00:06<00:00,  1.29s/it]

(19091, 16)


,compensation_status,accident_id,vehicle_id,fd_decision,circumstances,road_signs,road_surface,visibility,road_conditions,accident_date,address_id,circumstances_id,road_signs_id,road_surface_id,visibility_id,road_conditions_id
11123,VALIDATED_EIF,84752,138906.0,FG,First,NaN,Other,Clear day,Dry,2018-01-11 08:10:00+04:00,754074,2.0,nan,5.0,1.0,1.0
11124,VALIDATED_EIF,84752,138906.0,FG,Direct traffic,NaN,Other,Clear day,Dry,2018-01-11 08:10:00+04:00,754074,12.0,nan,5.0,1.0,1.0
13205,VALIDATED_EIF,101310,168010.0,FG,Third,NaN,Asphalt-concrete,Clear day,Dry,2018-06-21 12:30:00+04:00,910393,4.0,nan,1.0,1.0,1.0
13206,VALIDATED_EIF,101310,168010.0,FG,Rearrangement to right,NaN,Asphalt-concrete,Clear day,Dry,2018-06-21 12:30:00+04:00,910393,13.0,nan,1.0,1.0,1.0
13207,VALIDATED_EIF,101310,168010.0,FG,Marker lights,NaN,Asphalt-concrete,Clear day,Dry,2018-06-21 12:30:00+04:00,910393,25.0,nan,1.0,1.0,1.0


In [54]:
import json
with open('../data/processed/feature_decoding.json', 'w') as f:
    json.dump(l_category_decoding, f)

### Feature Engineering

In [55]:
num_cardinality = 0
dummy_col_names = []
for feature in cat_features:
    l_unique_vals = df[feature].unique()
    n_unique_vals = len(l_unique_vals)
    l_range_nunique = [i for i in range(n_unique_vals)] # noqa

    num_cardinality += n_unique_vals
    l_ind_value_name = [f"{feature}_{i}" for i in l_unique_vals]
    dummy_col_names = dummy_col_names + l_ind_value_name


print(f"Cardinality: {num_cardinality}")

Cardinality: 75


In [56]:
# One hot encoding
df_prep = pd.get_dummies(df, columns=cat_features + ['fd_decision'])

# daytime features
df_prep['hour'] = df_prep.accident_date.dt.hour
df_prep['day_of_month'] = df_prep.accident_date.dt.day
df_prep['month'] = df_prep.accident_date.dt.month
df_prep['year'] = df_prep.accident_date.dt.year

# accident level features
df_temp = df[['accident_id', 'vehicle_id']].groupby('accident_id').agg({"vehicle_id": ["nunique"]}).reset_index()
df_temp.columns = ["".join(col).strip().replace("nunique", "") for col in df_temp.columns.values]
dict_acc_veh_count = dict(zip(df_temp.accident_id, df_temp.vehicle_id))

df_prep['n_vehicles_left_in_accident'] = df_prep.accident_id.map(dict_acc_veh_count)

cols_to_drop = [f"{col}_id" for col in cat_features] + \
               [
    'compensation_status',
    'address_id',
    'accident_date',
    'accident_id'
               ]


# df_prep = df_prep.drop(cols_to_drop, axis=1).set_index('vehicle_id')
# print(df_prep.shape)
# df_prep.head()

In [60]:
df_prep.vehicle_id.value_counts()

154148.0    22
63517.0     22
60514.0     22
47888.0     20
125928      20
            ..
59685.0      1
106818.0     1
219487.0     1
137137.0     1
53538.0      1
Name: vehicle_id, Length: 9008, dtype: int64

In [58]:
from random import randint

def random_with_N_digits(n):
    range_start = 10**(n-1)
    range_end = (10**n)-1
    return randint(range_start, range_end)

[random_with_N_digits(6) for i in range(df_prep.shape[0])]

,compensation_status,accident_id,vehicle_id,accident_date,address_id,circumstances_id,road_signs_id,road_surface_id,visibility_id,road_conditions_id,...,road_conditions_Other,road_conditions_Snow,road_conditions_Wet,fd_decision_FG,fd_decision_NG,hour,day_of_month,month,year,n_vehicles_left_in_accident
11123,VALIDATED_EIF,84752,138906.0,2018-01-11 08:10:00+04:00,754074,2.0,nan,5.0,1.0,1.0,...,0,0,0,1,0,8,11,1,2018,1
11124,VALIDATED_EIF,84752,138906.0,2018-01-11 08:10:00+04:00,754074,12.0,nan,5.0,1.0,1.0,...,0,0,0,1,0,8,11,1,2018,1
13205,VALIDATED_EIF,101310,168010.0,2018-06-21 12:30:00+04:00,910393,4.0,nan,1.0,1.0,1.0,...,0,0,0,1,0,12,21,6,2018,2
13206,VALIDATED_EIF,101310,168010.0,2018-06-21 12:30:00+04:00,910393,13.0,nan,1.0,1.0,1.0,...,0,0,0,1,0,12,21,6,2018,2
13207,VALIDATED_EIF,101310,168010.0,2018-06-21 12:30:00+04:00,910393,25.0,nan,1.0,1.0,1.0,...,0,0,0,1,0,12,21,6,2018,2
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
18439,VALIDATED_EIF,188892,566531.0,2021-07-19 01:40:00+04:00,1968163,nan,nan,1.0,1.0,1.0,...,0,0,0,0,1,1,19,7,2021,2
18440,VALIDATED_EIF,188892,566529.0,2021-07-19 01:40:00+04:00,1968163,9.0,nan,1.0,1.0,1.0,...,0,0,0,1,0,1,19,7,2021,2
18441,VALIDATED_EIF,188892,566529.0,2021-07-19 01:40:00+04:00,1968163,11.0,nan,1.0,1.0,1.0,...,0,0,0,1,0,1,19,7,2021,2
18442,VALIDATED_EIF,188892,566529.0,2021-07-19 01:40:00+04:00,1968163,17.0,nan,1.0,1.0,1.0,...,0,0,0,1,0,1,19,7,2021,2


In [26]:
df_prep.to_csv('../data/processed/data_prepared.csv')

### Checkings

In [59]:
df_fil[df_fil.vehicle_id=='1396']

,compensation_status,accident_id,vehicle_id,fd_decision,circumstances,road_signs,road_surface,visibility,road_conditions,accident_date,address_id
10285,VALIDATED_EIF,073,1396,FG,5,nan,1,2,1,2020-09-13 20:10:00+04:00,8879
10286,VALIDATED_EIF,073,1396,FG,11,nan,1,2,1,2020-09-13 20:10:00+04:00,8879
10287,VALIDATED_EIF,073,1396,FG,15,nan,1,2,1,2020-09-13 20:10:00+04:00,8879
11187,VALIDATED_EIF,84977,1396,FG,nan,nan,1,1,1,2020-08-30 14:30:00+04:00,75
11192,VALIDATED_EIF,882,1396,FG,2,nan,1,1,1,2020-09-27 13:10:00+04:00,7379
11193,VALIDATED_EIF,882,1396,FG,12,nan,1,1,1,2020-09-27 13:10:00+04:00,7379


In [60]:
df_fil[df_fil.vehicle_id=='7']

,compensation_status,accident_id,vehicle_id,fd_decision,circumstances,road_signs,road_surface,visibility,road_conditions,accident_date,address_id
461,VALIDATED_EIF,18633,7,FG,3,nan,1,2,1,2020-02-09 21:20:00+04:00,1046
462,VALIDATED_EIF,18633,7,FG,11,nan,1,2,1,2020-02-09 21:20:00+04:00,1046
463,VALIDATED_EIF,18633,7,FG,31,nan,1,2,1,2020-02-09 21:20:00+04:00,1046
548,VALIDATED_EIF,19128,7,FG,2,nan,1,1,1,2020-02-11 12:25:00+04:00,1681
549,VALIDATED_EIF,19128,7,FG,12,nan,1,1,1,2020-02-11 12:25:00+04:00,1681
550,VALIDATED_EIF,19128,7,FG,,nan,1,1,1,2020-02-11 12:25:00+04:00,1681
1606,VALIDATED_EIF,27628,7,FG,3,nan,1,2,1,2020-03-01 23:30:00+04:00,218443
1607,VALIDATED_EIF,27628,7,FG,12,nan,1,2,1,2020-03-01 23:30:00+04:00,218443
1615,VALIDATED_EIF,27639,7,FG,2,nan,1,1,1,2020-03-02 15:45:00+04:00,218648
1616,VALIDATED_EIF,27639,7,FG,15,nan,1,1,1,2020-03-02 15:45:00+04:00,218648


In [57]:
df_fil[df_fil.index=='1396']

,circumstances_Alarm signal,circumstances_Backward motion,circumstances_Beginning of traffic,circumstances_Breaking,circumstances_Detour,circumstances_Direct traffic,circumstances_Fifth,circumstances_First,circumstances_Fog lights,circumstances_Fourth,...,road_conditions_Other,road_conditions_Snow,road_conditions_Wet,fd_decision_FG,fd_decision_NG,hour,day_of_month,month,year,n_vehicles_left_in_accident
vehicle_id,,,,,,,,,,,,,,,,,,,,,
1396,0,0,0,0,0,0,0,0,0,1,...,0,0,0,1,0,20,13,9,2020,2
1396,0,0,1,0,0,0,0,0,0,0,...,0,0,0,1,0,20,13,9,2020,2
1396,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,20,13,9,2020,2
1396,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,14,30,8,2020,2
1396,0,0,0,0,0,0,0,1,0,0,...,0,0,0,1,0,13,27,9,2020,7
1396,0,0,0,0,0,1,0,0,0,0,...,0,0,0,1,0,13,27,9,2020,7


In [61]:
df_fil

,compensation_status,accident_id,vehicle_id,fd_decision,circumstances,road_signs,road_surface,visibility,road_conditions,accident_date,address_id
0,VALIDATED_EIF,16141,26245,NG,nan,nan,1,2,1,2020-02-01 01:30:00+04:00,141147
1,VALIDATED_EIF,16141,26244,FG,2,nan,1,2,1,2020-02-01 01:30:00+04:00,141147
2,VALIDATED_EIF,16141,26244,FG,12,nan,1,2,1,2020-02-01 01:30:00+04:00,141147
3,VALIDATED_EIF,16141,26244,FG,35,nan,1,2,1,2020-02-01 01:30:00+04:00,141147
4,VALIDATED_EIF,16169,26324,NG,2,nan,1,1,2,2020-02-01 18:00:00+04:00,141548
...,...,...,...,...,...,...,...,...,...,...,...
18439,VALIDATED_EIF,188892,566531,NG,nan,nan,1,1,1,2021-07-19 01:40:00+04:00,1968163
18440,VALIDATED_EIF,188892,566529,FG,9,nan,1,1,1,2021-07-19 01:40:00+04:00,1968163
18441,VALIDATED_EIF,188892,566529,FG,11,nan,1,1,1,2021-07-19 01:40:00+04:00,1968163
18442,VALIDATED_EIF,188892,566529,FG,17,nan,1,1,1,2021-07-19 01:40:00+04:00,1968163
